In [2]:
install.packages("forecast")

Installing package into ‘/home/shhwang/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

also installing the dependencies ‘quadprog’, ‘fracdiff’, ‘lmtest’, ‘timeDate’, ‘tseries’, ‘urca’




In [61]:
library(dplyr)
library(stats)
library(grDevices)#for plot font
library(forecast)
library(onlineFDR)

### Data Generation Step

In [53]:
data<-read.csv("~/SAST_covid/owid-covid-data.csv")

In [54]:
data$date<-as.Date(data$date)
data$weekdays<-weekdays(data$date)
sunday<-which(data$weekdays=='Sunday')

In [55]:
data_of = function(loc, name ){
  dat = data[data$location==loc,]
  new_cases = dat$total_cases-c(dat$new_cases[1],dat$total_cases[-nrow(dat)])
  new_cases = ifelse(is.na(dat$total_cases)&is.na(dat$new_cases),0, ifelse(is.na(dat$new_cases), new_cases, dat$new_cases))
  dat$new_cases <- abs(new_cases)
  
  new_vacc = ifelse(is.na(dat$new_vaccinations), 0, dat$new_vaccinations)
  dat$new_vaccinations<-new_vacc
  
  
  new_cases_smoothed = sapply(6:nrow(dat), function(x) ifelse(is.na(dat$new_cases_smoothed[x]), mean(dat$new_cases[(x-5):x]) ,dat$new_cases_smoothed[x]))
  dat$new_cases_smoothed[6:nrow(dat)] = new_cases_smoothed
  dat = dat[!is.na(dat$new_cases_smoothed),]
  
  dat_N = dat$population[1]
  dat_S = dat_N-cumsum(dat$new_cases_smoothed)
  dat_I = n_rolling2(dat$new_cases, 7)
  assign(name, dat, envir = globalenv())
  assign(paste0(name,'_N'), dat_N, envir = globalenv())
  assign(paste0(name, '_S'), dat_S, envir = globalenv())
  assign(paste0(name, '_I'), dat_I, envir = globalenv())
  assign(paste0(name, '_J'), dat$new_cases, envir = globalenv())
  assign(paste0(name, '_V'), dat$new_vaccinations, envir = globalenv())
  
  
  print(paste0(name,' has been extracted'))
}

In [8]:
if(.Platform$OS.type == "unix") { # includes Mac
  
  utils::str( quartzFonts() ) # a list(serif = .., sans = .., mono = ..)
  quartzFonts("mono") # the list(mono = ..) sublist of quartzFonts()
  ## Not run: 
  ## for East Asian locales you can use something like
  quartzFonts(sans = quartzFont(c("Helvetica", "Helvetica-Bold" ,"Helvetica-Oblique" ,"Helvetica-BoldOblique")),
              serif = quartzFont(c("Times-Roman", "Times-Bold", "Times-Italic", "Times-BoldItalic")))
  ## since the default fonts may well not have the glyphs needed
  
  ## End(Not run)
}

List of 3
 $ serif: chr [1:4] "Times-Roman" "Times-Bold" "Times-Italic" "Times-BoldItalic"
 $ sans : chr [1:4] "Helvetica" "Helvetica-Bold" "Helvetica-Oblique" "Helvetica-BoldOblique"
 $ mono : chr [1:4] "Courier" "Courier-Bold" "Courier-Oblique" "Courier-BoldOblique"


### Useful Functions

In [9]:
mid_rolling = function(J){
    rolling_case = sapply(1:length(J), function(x){
        m = pmax(x-3, 1)
        M = pmin(x+3, length(J))
        d = M-m+1
        res = mean(J[m:M])
        return(res)
    })
    return(rolling_case)
}

In [10]:
n_rolling2 = function(case, interval = 7){

  rolling_case = sapply(1:length(case), function(x) mean(case[x:max(1,(x-interval+1))]))
  
  return(rolling_case)
}

In [11]:
clfdr_cutoff = function(clfdr, prev_clfdr = 0.99, alpha = 0.05){
  sorted = sort(clfdr)
  clfdr_mean = sapply(1:length(clfdr), function(x) mean(sorted[1:x]))
  loc = which.max(clfdr_mean[clfdr_mean<alpha])
  cutoff = ifelse(length(loc)==0, prev_clfdr, sorted[loc])
  return(cutoff)
}
online_fdr2 = function(clfdr, window=30, alpha=0.05){
  
  win = clfdr[1 : window]
  
  R = sum(rej1[1:window]) # number of rejection
  cusum_clfdr = sum(win[rej1[1:window]])
  rej2 = logical(length(clfdr))
  rej2[1:window]<-rej1[1:window]
  cutoff = clfdr_cutoff(win)
  
  for(i in (window+1):length(clfdr)){
    new_clfdr = clfdr[i]
    win = c(win[-1], new_clfdr)
    
    local_res = rev(offline_fdr(win, alpha))[1]
    
    if(local_res & new_clfdr<0.99){
      if(cusum_clfdr+new_clfdr <= (R+1)*alpha){
        R = R+1
        cusum_clfdr = cusum_clfdr + new_clfdr
        rej2[i]<-T
      }
    }
    cutoff = clfdr_cutoff(win, cutoff, alpha)
  }
  
  return(rej2)
}
offline_fdr = function(clfdr, alpha = 0.05){
  clfdr_sorted = sort(clfdr)
  find_max = sapply(1:length(clfdr), function(x) mean(clfdr_sorted[1:x]))
  ss = sum(find_max<alpha)
  if(ss==0){
    return(rep(F, length(clfdr)))
  }else{
    loc = max(which(find_max<alpha))
    max_clfdr = clfdr_sorted[loc]
    return(clfdr<=max_clfdr)
  }
}


online_fdr = function(clfdr, rej1, window = 30, alpha = 0.05){
  win = clfdr[1 : window]
  
  R = sum(rej1[1:window]) # number of rejection
  cusum_clfdr = sum(win[rej1[1:window]])
  rej2 = logical(length(clfdr))
  rej2[1:window]<-rej1[1:window]
  
  for(i in (window+1):length(clfdr)){
    new_clfdr = clfdr[i]
    
    if(rej1[i]){
      if(cusum_clfdr+new_clfdr <= (R+1)*alpha){
        R = R+1
        cusum_clfdr = cusum_clfdr + new_clfdr
        rej2[i]<-T
      }
    }
  }
  
  return(rej2)
}

### Model 1

In [12]:
m1_state3 = function(loc,train = 50, r = c(0.9, 1, 1.1), pi = c(0.001, 0.99, 0.009), A=matrix(0.01,3,3)+diag(0.98,3), start = 50){
    rr = r

    J = get(paste0(loc,'_J'))
    J = J[!is.na(J)]+1
    R0 = get(loc)$reproduction_rate
  
    I = n_rolling2(J, 7)
  
    st = which(J>0)[1]+start
    end = st + train -1
  
    J = J[st:end]
    I = I[(st-1):(end-1)]
    time_series = ts(log(J/I), frequency = 7)
    ma = stl(time_series, 'periodic')
    Eta = exp(ma$time.series[,1])
    I = I*Eta
  
  ####alpha
  
  get_log_alpha_ratio = function(r, pi, A){
    lar23 <- numeric(train) 
    lar21 <- numeric(train) 
    
    lf = function(n, rn) dpois(J[n], r[rn]*I[n], log=T)-dpois(J[n], r[2]*I[n], log=T)
    
    lar23[1]<-lf(1,3)+log(pi[3])-log(pi[2]) 
    lar21[1]<-lf(1,1)+log(pi[1])-log(pi[2]) 
    diff<-c(lar23[1], lar21[1], lar23[1]-lar21[1])
    
    
    for(i in 2:train){
      loglr23 = lf(i,3)
      loglr21 = lf(i,1)
      
      if(diff[1]>709){
        if(diff[3]>709){
          logarr23 = log(A[3,3])-log(A[3,2])
          logarr21 = log(A[3,1])-log(A[3,2])
        }else{
          logarr23 = log(A[1,3]+A[3,3]*exp(diff[3]))-log(A[1,2]+A[3,2]*exp(diff[3]))
          logarr21 = log(A[1,1]+A[3,1]*exp(diff[3]))-log(A[1,2]+A[3,2]*exp(diff[3]))
        }
      }else if(diff[2]>709){
        if(diff[3]>709){
          logarr23 = log(A[3,3])-log(A[3,2])
          logarr21 = log(A[3,1])-log(A[3,2])
        }else{
          logarr23 = log(A[1,3]+A[3,3]*exp(diff[3]))-log(A[1,2]+A[3,2]*exp(diff[3]))
          logarr21 = log(A[1,1]+A[3,1]*exp(diff[3]))-log(A[1,2]+A[3,2]*exp(diff[3]))
        }
      }else if(max(diff[1:2])<(-709)){
        logarr23 = log(A[2,3])-log(A[2,2])
        logarr21 = log(A[2,1])-log(A[2,2])
      }else{
        ar23 = lar23[i-1]
        ar21 = lar21[i-1]
        A13 = A[1,3]*exp(ar21)
        A23 = A[2,3]
        A33 = A[3,3]*exp(ar23)
        
        A12 = A[1,2]*exp(ar21)
        A22 = A[2,2]
        A32 = A[3,2]*exp(ar23)
        
        A11 = A[1,1]*exp(ar21)
        A21 = A[2,1]
        A31 = A[3,1]*exp(ar23)
        
        logarr23 = log(A13+A23+A33)-log(A12+A22+A32)
        logarr21 = log(A11+A21+A31)-log(A12+A22+A32)
      }
      
      lar23[i]<-(loglr23+logarr23)
      lar21[i]<-(loglr21+logarr21)
      
      diff<-c(lar23[i], lar21[i], lar23[i]-lar21[i])
    }
    
    return(cbind(lar21, lar23))
  }
  
  get_log_beta_ratio = function(r, A){
    lbr23 <- numeric(train)
    lbr21 <- numeric(train)
    
    lf = function(n, rn) dpois(J[n], r[rn]*I[n], log=T)-dpois(J[n], r[2]*I[n], log=T)
    
    loglr23 = lf(train, 3)
    loglr21 = lf(train, 1)
    
    diff = c(loglr23, loglr21, loglr23-loglr21)
    if(diff[2]>709){
      if(diff[3]>709){
        lbr23[train]<-log(A[3,3])-log(A[2,1]+A[2,3])
        lbr21[train]<-log(A[1,3])-log(A[2,1]+A[2,3])
      }else{
        lbr23[train]<-log(A[3,1]+A[3,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
        lbr21[train]<-log(A[1,1]+A[1,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
      }
    }else if(diff[1]>709){
      if(diff[3]>709){
        lbr23[train] = log(A[3,3])-log(A[2,2])
        lbr21[train] = log(A[1,1])-log(A[1,2])
      }else{
        lbr23[train]<-log(A[3,1]+A[3,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
        lbr21[train]<-log(A[1,1]+A[1,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
      }
    }else{
      
      lbr23[train] = log(A[3,1]*exp(loglr21)+A[3,2]+A[3,3]*exp(loglr23))-log(A[2,1]*exp(loglr21)+A[2,2]+A[2,3]*exp(loglr23))
      lbr21[train] = log(A[1,1]*exp(loglr21)+A[1,2]+A[1,3]*exp(loglr23))-log(A[2,1]*exp(loglr21)+A[2,2]+A[2,3]*exp(loglr23))
    }
    
    
    for(i in (train-1):1){
      loglr23 = lf(i, 3)
      loglr21 = lf(i, 1)
      
      br23 = loglr23+lbr23[i+1]
      br21 = loglr21+lbr21[i+1]
      
      diff = c(br23, br21, br23-br21)
      
      if(diff[2]>709){
        if(diff[3]>709){
          lbr23[i]<-log(A[3,3])-log(A[2,1]+A[2,3])
          lbr21[i]<-log(A[1,3])-log(A[2,1]+A[2,3])
        }else{
          lbr23[i]<-log(A[3,1]+A[3,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
          lbr21[i]<-log(A[1,1]+A[1,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
        }
      }else if(diff[1]>709){
        if(diff[3]>709){
          lbr23[i] = log(A[3,3])-log(A[2,2])
          lbr21[i] = log(A[1,1])-log(A[1,2])
        }else{
          lbr23[i]<-log(A[3,1]+A[3,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
          lbr21[i]<-log(A[1,1]+A[1,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
        }
      }else{
        
        lbr23[i] = log(A[3,1]*exp(br21)+A[3,2]+A[3,3]*exp(br23))-log(A[2,1]*exp(br21)+A[2,2]+A[2,3]*exp(br23))
        lbr21[i] = log(A[1,1]*exp(br21)+A[1,2]+A[1,3]*exp(br23))-log(A[2,1]*exp(br21)+A[2,2]+A[2,3]*exp(br23))
      }
      
    }
    return(cbind(lbr21, lbr23))
  }
  
  get_pij = function(logar, logbr, r, A){
    #logar = result from get_log_alpha_ratio
    #logbr = result from get_log_beta_ratio
    a2 = rep(0, train-1)
    a1 = logar[-train,1]
    a3 = logar[-train,2]
    
    b2 = rep(0, train-1)
    b1 = logbr[-1,1]
    b3 = logbr[-1,2]
    
    It = I[-1]
    Jt = J[-1]
    p11_p = a1+b1+log(A[1,1])+dpois(Jt, r[1]*It, log = T)
    p12_p = a1+b2+log(A[1,2])+dpois(Jt, r[2]*It, log = T)
    p13_p = a1+b3+log(A[1,3])+dpois(Jt, r[3]*It, log = T)
    
    p21_p = a2+b1+log(A[2,1])+dpois(Jt, r[1]*It, log = T)
    p22_p = a2+b2+log(A[2,2])+dpois(Jt, r[2]*It, log = T)
    p23_p = a2+b3+log(A[2,3])+dpois(Jt, r[3]*It, log = T)
    
    p31_p = a3+b1+log(A[3,1])+dpois(Jt, r[1]*It, log = T)
    p32_p = a3+b2+log(A[3,2])+dpois(Jt, r[2]*It, log = T)
    p33_p = a3+b3+log(A[3,3])+dpois(Jt, r[3]*It, log = T)
    
    pp = cbind(p11_p, p12_p, p13_p, 
               p21_p, p22_p, p23_p,
               p31_p, p32_p, p33_p)
    
    
    ppp = apply(pp,1,function(xx){
      if(all(is.infinite(xx))){
        xx<-ifelse(xx>0, exp(708), exp(-708))
      }
      if(max(xx)>708){
        mgap = max(xx)-708
        xx = xx-mgap
      }else if(min(xx)<(-708)){
        mgap = 708-max(xx)
        xx = xx+mgap
      }
      
      if(max(xx)>708){
        mgap = max(xx)-708
        xx = xx-mgap
      }
      
      return(exp(xx)/sum(exp(xx)))
    })
    ppp = t(ppp)
    
    for(k in c(1,4,7)){
      if(sum(ppp[,k:(k+2)])<exp(-708)){
        pp2 = pp[,k:(k+2)]
        if(max(pp2)>708){
          mgap = max(pp2)-708
          pp2 = pp2-mgap
        }else if(min(pp2)<(-708)){
          mgap = 708-max(pp2)
          pp2 = pp2+mgap
        }
        
        if(max(pp2)>708){
          mgap = max(pp2)-708
          pp2 = pp2-mgap
        }
        ppp2 = apply(pp2,1,function(xx){
          res = exp(xx)/sum(exp(xx))
          if(all(is.nan(res))){
            res = rep(0,3)
          }
          return(res)
        })
        ppp2 = t(ppp2)
        ppp[,k:(k+2)]<-ppp2
      }
    }
    
    
    return(ppp)
  }
  
  gap = 1000
  it.num = 1
  
  while((it.num<1000 & gap>0.001)){
    
    logar = get_log_alpha_ratio(r, pi, A)
    logbr = get_log_beta_ratio(r, A)
    
    pb = logar+logbr
    
    ps = apply(pb, 1, function(x){
      diff = x[1]-x[2]
      if(x[1]>709){
        if(diff>709){
          y = c(1,0,0)
        }else{
          y = c(exp(diff)/(1+exp(diff)),0,1/(1+exp(diff)))
        }
      }else if(x[2]>709){
        if(diff<(-709)){
          y = c(0,0,1)
        }else if(diff>709){
          y = c(1,0,0)
        }else{
          yk = c(x[1],0,x[2])-min(x)
          y = exp(yk)/sum(exp(yk))
        }
      }else{
        y = exp(c(x[1],0,x[2]))/sum(exp(c(0,x)))
      }
    })
    
    ps = t(ps)
    p1 = ps[,1]
    p2 = ps[,2]
    p3 = ps[,3]
    
    pij = get_pij(logar,logbr,r,A)
    
    newA = matrix(0,3,3)
    cols = colSums(pij)
    
    newA[1,1:3] = cols[1:3]/sum(cols[1:3])
    newA[2,1:3] = cols[4:6]/sum(cols[4:6])
    newA[3,1:3] = cols[7:9]/sum(cols[7:9])
    
    hurdle = 1/train
    if(any(newA<hurdle)){
      locs = which(newA<hurdle, arr.ind = T)
      for(k in 1:nrow(locs)){
        nr = locs[k,1]
        nc = locs[k,2]
        nm = which.max(newA[nr,])
        newA[nr,nc]<-hurdle
        newA[nr, nm]<-newA[nr, nm]-hurdle
      }
    }
    
    newpi = c(p1[1], p2[1], p3[1])/(p1[1]+p2[1]+p3[1])
    newpi = ifelse(newpi==0,exp(-706), ifelse(newpi==1, 1-2*exp(-706), newpi))
    
    r1 = ifelse(sum(p1)>0.5, weighted.mean(J, p1)/weighted.mean(I, p1), 0.99)
    r3 = ifelse(sum(p3)>0.5,weighted.mean(J, p3)/weighted.mean(I, p3),1.01)
    r1<-ifelse(r1>1, 0.99, r1)
    r3<-ifelse(r3<1, 1.01, r3)
    r3<-ifelse(r3>1.5, 1.5, r3)
    newr = c(r1,1, r3)
    
    #maxm1 = optimize(function(x) maxf(x,p1),c(0.00001,0.999999))$minimum
    #maxm3 = optimize(function(x) maxf(-x,p3),c(0.00001,0.999999))$minimum
    #newr = c(1-maxm1, 1, 1+maxm3)
    
    Agap = max(abs(A-newA))
    pigap = max(abs(pi-newpi))
    rgap = max(abs(r-newr))
    
    gap = max(Agap, pigap, rgap)
    
    if(!is.nan(gap)){
      A <- newA
      pi <- newpi
      r <- newr
    }
    
    it.num = it.num+1
  }
  
  return(list(r = r, pi = pi, A = A, p1 = p1, p2 = p2, p3 = p3, it = it.num, Ieta = I))
}


In [40]:
online_fdr_ontime1 = function(loc, r=c(0.95,1,1.05), pi=c(0.01,0.98,0.01), A=matrix(0.01,3,3)+diag(0.98,3), window = 30, start = 50){
  
  train = window
  J = get(paste0(loc,'_J'))
  J = J[!is.na(J)]; J[J==0] <- 1
  I = n_rolling2(J,7)
  st = which(I>0)[1]+start
  end = st + 365*2
  
  J = J[st:end]
  I = I[(st-1):(end-1)]
  
  dt = get(loc)$date[st:end]
  #mut_dt = dt %in% as.Date(c('2020-12-01', '2020-06-20', '2020-09-24','2020-10-05', '2021-11-09'))
  #MUT = mut_dt[st:end]
  
  m1_res = m1_state3(loc,train=window, r, pi, A, start = start)
  
  p1 = m1_res$p1
  p2 = m1_res$p2
  p3 = m1_res$p3
  
  r = m1_res$r

    Ieta = numeric(length(J))
    Ieta[1:train] <- m1_res$Ieta[1:train]
  r_track = matrix(NA, nc = 3, nr = length(J))
  r_track[1:train,]<-rep(r,each = train)
  
  A_track = matrix(NA, nc = 9, nr = nrow(r_track))
  A_track[1:train,]<-rep(m1_res$A, each = train)
  
  blfdr = matrix(NA, nr = length(J), nc = 3)
  blfdr[1:train, 1]<-p1
  blfdr[1:train, 2]<-p2
  blfdr[1:train, 3]<-p3
  
  rej1 = logical(length(J))
  rej1[1:train] = offline_fdr(blfdr[1:train,1]+blfdr[1:train,2])
  
  maxl = (nrow(r_track)-window)
  for(i in 1:maxl){
    training = window
    rr = apply(r_track, 2, function(x) mean(x, na.rm = T))
    
    m1_res = m1_state3(loc,train=training,r=r, pi = pi, start = start+i)

               Ieta[train+i] <- m1_res$Ieta[train]
    r = m1_res$r; r_track[i+window,]<-r
    A = m1_res$A; A_track[i+window,]<-A 
    pi = c(m1_res$p1[2], m1_res$p2[2], m1_res$p3[2])
    pi = ifelse(pi==0,exp(-706), ifelse(pi==1, 1-2*exp(-706), pi))
    
    blfdr[i+window,] <- c(m1_res$p1[train], m1_res$p2[train], m1_res$p3[train])
    clfdr = m1_res$p1+m1_res$p2
    rej1[i+window] = offline_fdr(clfdr)[window]
    

  }
  
  
  rej1[1:window]<-F
  rseq = r_track[,3]-1
  clfdr = rowSums(blfdr[,1:2])
  rej2 = online_fdr(clfdr, rej1, window)
  
  rej = rej1  & rej2
  
  
  return(list(clfdr=clfdr, rej = rej, r_track = r_track, A_track = A_track, blfdr = blfdr, J = J, date = dt, Ieta = Ieta))
}

### Model 2
For model 2, J_t follows Negative binomial distribution.
* m2_state3 : for each window
* online_fdr_ontime2: test as if online with given offline data

#### m2_state 

In [14]:

m2_state3 = function(loc,train = 50, a, b, pi = c(0.01, 0.98,0.01), A = matrix(c(0.7,0.05,0.3,0.2,0.9,0.1,0.1,0.05,0.6),3,3), start = 50){
  
  aa = a
  rr = c((aa/aa[2])[1],1,(aa/aa[2])[3])
  
    J = get(paste0(loc,'_J'))
    J = J[!is.na(J)]+1
    
    I = n_rolling2(J, 7)
  
    st = which(J>0)[1]+start
    end = st + train -1
  
    J = J[st:end]
    I = I[(st-1):(end-1)]
    time_series = ts(log(J/I), frequency = 7)
    ma = stl(time_series, 'periodic')
    Eta = exp(ma$time.series[,1])
    I = I*Eta

  
  ####alpha
  
  get_log_alpha_ratio = function(a, b, pi, A){
    lar23 <- numeric(train) 
    lar21 <- numeric(train) 
    
    lf = function(n, an) dnbinom(J[n], a[an], b/(I[n]+b), log = T)-dnbinom(J[n], a[2],  b/(I[n]+b), log = T)
    
    
    lar23[1]<-lf(1,3) + log(pi[3])-log(pi[2])
    lar21[1]<-lf(1,1) + log(pi[1])-log(pi[2])
    diff<-c(lar23[1], lar21[1], lar23[1]-lar21[1])
    
    for(i in 2:train){
      loglr23 = lf(i,3)
      loglr21 = lf(i,1)
      
      if(diff[1]>709){
        if(diff[3]>709){
          logarr23 = log(A[3,3])-log(A[3,2])
          logarr21 = log(A[3,1])-log(A[3,2])
        }else{
          logarr23 = log(A[1,3]+A[3,3]*exp(diff[3]))-log(A[1,2]+A[3,2]*exp(diff[3]))
          logarr21 = log(A[1,1]+A[3,1]*exp(diff[3]))-log(A[1,2]+A[3,2]*exp(diff[3]))
        }
      }else if(diff[2]>709){
        if(diff[3]>709){
          logarr23 = log(A[3,3])-log(A[3,2])
          logarr21 = log(A[3,1])-log(A[3,2])
        }else{
          logarr23 = log(A[1,3]+A[3,3]*exp(diff[3]))-log(A[1,2]+A[3,2]*exp(diff[3]))
          logarr21 = log(A[1,1]+A[3,1]*exp(diff[3]))-log(A[1,2]+A[3,2]*exp(diff[3]))
        }
      }else if(max(diff[1:2])<(-709)){
        logarr23 = log(A[2,3])-log(A[2,2])
        logarr21 = log(A[2,1])-log(A[2,2])
      }else{
        ar23 = lar23[i-1]
        ar21 = lar21[i-1]
        A13 = A[1,3]*exp(ar21)
        A23 = A[2,3]
        A33 = A[3,3]*exp(ar23)
        
        A12 = A[1,2]*exp(ar21)
        A22 = A[2,2]
        A32 = A[3,2]*exp(ar23)
        
        A11 = A[1,1]*exp(ar21)
        A21 = A[2,1]
        A31 = A[3,1]*exp(ar23)
        
        logarr23 = log(A13+A23+A33)-log(A12+A22+A32)
        logarr21 = log(A11+A21+A31)-log(A12+A22+A32)
      }
      
      lar23[i]<-(loglr23+logarr23)
      lar21[i]<-(loglr21+logarr21)
      
      diff<-c(lar23[i], lar21[i], lar23[i]-lar21[i])
    }
    
    return(cbind(lar21, lar23))
  }
  
  get_log_beta_ratio = function(a, b, A){
    lbr23 <- numeric(train)
    lbr21 <- numeric(train)
    
    lf = function(n, an) dnbinom(J[n], a[an], b/(I[n]+b), log = T)-dnbinom(J[n], a[2],  b/(I[n]+b), log = T)
    
    loglr23 = lf(train, 3)
    loglr21 = lf(train, 1)
    
    diff = c(loglr23, loglr21, loglr23-loglr21)
    if(diff[2]>709){
      if(diff[3]>709){
        lbr23[train]<-log(A[3,3])-log(A[2,1]+A[2,3])
        lbr21[train]<-log(A[1,3])-log(A[2,1]+A[2,3])
      }else{
        lbr23[train]<-log(A[3,1]+A[3,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
        lbr21[train]<-log(A[1,1]+A[1,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
      }
    }else if(diff[1]>709){
      if(diff[3]>709){
        lbr23[train] = log(A[3,3])-log(A[2,2])
        lbr21[train] = log(A[1,1])-log(A[1,2])
      }else{
        lbr23[train]<-log(A[3,1]+A[3,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
        lbr21[train]<-log(A[1,1]+A[1,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
      }
    }else{
      
      lbr23[train] = log(A[3,1]*exp(loglr21)+A[3,2]+A[3,3]*exp(loglr23))-log(A[2,1]*exp(loglr21)+A[2,2]+A[2,3]*exp(loglr23))
      lbr21[train] = log(A[1,1]*exp(loglr21)+A[1,2]+A[1,3]*exp(loglr23))-log(A[2,1]*exp(loglr21)+A[2,2]+A[2,3]*exp(loglr23))
    }
    
    
    for(i in (train-1):1){
      loglr23<-lf(i,3)
      loglr21<-lf(i,1)
      
      br23 = loglr23+lbr23[i+1]
      br21 = loglr21+lbr21[i+1]
      
      diff = c(br23, br21, br23-br21)
      
      if(diff[2]>709){
        if(diff[3]>709){
          lbr23[i]<-log(A[3,3])-log(A[2,1]+A[2,3])
          lbr21[i]<-log(A[1,3])-log(A[2,1]+A[2,3])
        }else{
          lbr23[i]<-log(A[3,1]+A[3,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
          lbr21[i]<-log(A[1,1]+A[1,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
        }
      }else if(diff[1]>709){
        if(diff[3]>709){
          lbr23[i] = log(A[3,3])-log(A[2,2])
          lbr21[i] = log(A[1,1])-log(A[1,2])
        }else{
          lbr23[i]<-log(A[3,1]+A[3,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
          lbr21[i]<-log(A[1,1]+A[1,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
        }
      }else{
        
        lbr23[i] = log(A[3,1]*exp(br21)+A[3,2]+A[3,3]*exp(br23))-log(A[2,1]*exp(br21)+A[2,2]+A[2,3]*exp(br23))
        lbr21[i] = log(A[1,1]*exp(br21)+A[1,2]+A[1,3]*exp(br23))-log(A[2,1]*exp(br21)+A[2,2]+A[2,3]*exp(br23))
      }
      
    }
    return(cbind(lbr21, lbr23))
  }
  
  get_pij = function(logar, logbr, a, b, A){
    
    a2 = rep(0, train-1)
    a1 = (logar[-train,1])
    a3 = (logar[-train,2])
    
    b2 = rep(0, train-1)
    b1 = (logbr[-1,1])
    b3 = (logbr[-1,2])
    
    Jt = J[-1]
    It = I[-1]
    
    lnb = function(an) dnbinom(Jt, a[an], b/(It+b))
    
    p11_p = a1+b1+log(A[1,1])+lnb(1)
    p12_p = a1+b2+log(A[1,2])+lnb(2)
    p13_p = a1+b3+log(A[1,3])+lnb(3)
    
    p21_p = a2+b1+log(A[2,1])+lnb(1)
    p22_p = a2+b2+log(A[2,2])+lnb(2)
    p23_p = a2+b3+log(A[2,3])+lnb(3)
    
    p31_p = a3+b1+log(A[3,1])+lnb(1)
    p32_p = a3+b2+log(A[3,2])+lnb(2)
    p33_p = a3+b3+log(A[3,3])+lnb(3)
    
    pp = cbind(p11_p, p12_p, p13_p, 
               p21_p, p22_p, p23_p,
               p31_p, p32_p, p33_p)
    
    
    ppp = apply(pp,1,function(xx){
      if(all(is.infinite(xx))){
        xx<-ifelse(xx>0, exp(708), exp(-708))
      }
      if(max(xx)>708){
        mgap = max(xx)-708
        xx = xx-mgap
      }else if(min(xx)<(-708)){
        mgap = 708-max(xx)
        xx = xx+mgap
      }
      
      if(max(xx)>708){
        mgap = max(xx)-708
        xx = xx-mgap
      }
      
      return(exp(xx)/sum(exp(xx)))
    })
    ppp = t(ppp)
    
    for(k in c(1,4,7)){
      if(sum(ppp[,k:(k+2)])<exp(-708)){
        pp2 = pp[,k:(k+2)]
        if(max(pp2)>708){
          mgap = max(pp2)-708
          pp2 = pp2-mgap
        }else if(min(pp2)<(-708)){
          mgap = 708-max(pp2)
          pp2 = pp2+mgap
        }
        
        if(max(pp2)>708){
          mgap = max(pp2)-708
          pp2 = pp2-mgap
        }
        ppp2 = apply(pp2,1,function(xx){
          res = exp(xx)/sum(exp(xx))
          if(all(is.nan(res))){
            res = rep(0,3)
          }
          return(res)
        })
        ppp2 = t(ppp2)
        ppp[,k:(k+2)]<-ppp2
      }
    }
    
    
    return(ppp)
  }
  
  gap = 1000
  it.num = 1
  
  while((it.num<100 & gap>0.01)){
    
    logar = get_log_alpha_ratio(a, b, pi, A)
    logbr = get_log_beta_ratio(a, b, A)
    
    pb = logar+logbr
    
    ps = apply(pb, 1, function(x){
      diff = x[1]-x[2]
      if(x[1]>709){
        if(diff>709){
          y = c(1,0,0)
        }else{
          y = c(exp(diff)/(1+exp(diff)),0,1/(1+exp(diff)))
        }
      }else if(x[2]>709){
        if(diff<(-709)){
          y = c(0,0,1)
        }else if(diff>709){
          y = c(1,0,0)
        }else{
          y = c(exp(diff)/(1+exp(diff)),0,1/(1+exp(diff)))
        }
      }else{
        y = exp(c(x[1],0,x[2]))/sum(exp(c(0,x)))
      }
    })
    
    ps = t(ps)
    p1 = ps[,1]
    p2 = ps[,2]
    p3 = ps[,3]
    
    
    pij = get_pij(logar,logbr,a, b, A)
    
    newA = matrix(0,3,3)
    cols = colSums(pij)
    
    newA[1,1:3] = cols[1:3]/sum(cols[1:3])
    newA[2,1:3] = cols[4:6]/sum(cols[4:6])
    newA[3,1:3] = cols[7:9]/sum(cols[7:9])

    
    newpi = c(p1[1], p2[1], p3[1])
    newpi = ifelse(newpi==0,exp(-706), ifelse(newpi==1, 1-2*exp(-706), newpi))
    
    
    maxf = function(a,b,p){
      res = lgamma(a+J)-lgamma(a)+a*log(b)-a*log(b+I)
      return(sum(p*res))
    }

    willest = apply(ps,2,function(x) sum(x)>1)
    

    newb = ifelse(willest[2],optimize(function(x) -maxf(a[1], x, p1)-maxf(x,x,p2)-maxf(a[3],x,p3), c(a[1]*0.9, a[3]*1.1))$minimum, b)
    newa1 = ifelse(willest[1],optimize(function(x) -maxf(x,newb,p1), c(0.5, newb*0.99))$minimum, a[1])
    newa3 = ifelse(willest[3], optimize(function(x) -maxf(x,newb,p3), c(newb*1.01, 1.5*newb))$minimum, a[3])
    
    mgap = 10
    kk = 1
    while(mgap>0.01 & kk<50){
        newb = ifelse(willest[2],optimize(function(x) -maxf(a[1], x, p1)-maxf(x,x,p2)-maxf(a[3],x,p3), c(a[1]*0.9, a[3]*1.1))$minimum, b)
        newa1 = ifelse(willest[1],optimize(function(x) -maxf(x,newb,p1), c(0.5, newb*0.99))$minimum, a[1])
        newa3 = ifelse(willest[3], optimize(function(x) -maxf(x,newb,p3), c(newb*1.01, 1.5*newb))$minimum, a[3])

        mgap2 = abs(b-newb)
        if(mgap2>mgap){
            kk= 50
            newb = b
            newa1 = a[1]
            newa3 = a[3]
        }else{
            kk <- kk+1
            mgap = mgap2
        }
    }

    
    
    newa = c(newa1, newb, newa3)
    
    Agap = max(abs(A-newA))
    pigap = max(abs(pi-newpi))
    abgap = abs(b-newb)
    
    gap = max(Agap, pigap, abgap)
    
    if(!is.nan(gap)){
      A <- newA
      pi <- newpi
      a <- newa
      b <- newb
    }

    it.num = it.num+1
  }
  return(list(a = a, b=b, pi = pi, A = A, p1 = p1, p2 = p2, p3 = p3, it.num = it.num, Ieta = I))
}


In [15]:

m2_state3_test = function(loc,train = 50, a, b, pi = c(0.01, 0.98,0.01), A = matrix(c(0.7,0.05,0.3,0.2,0.9,0.1,0.1,0.05,0.6),3,3), start = 50){
  
  aa = a
  rr = c((aa/aa[2])[1],1,(aa/aa[2])[3])
  
    J = get(paste0(loc,'_J'))
    J = J[!is.na(J)]+1
    
    I = n_rolling2(J, 7)
  
    st = which(J>0)[1]+start
    end = st + train -1
  
    J = J[st:end]
    I = I[(st-1):(end-1)]
    time_series = ts(log(J/I), frequency = 7)
    ma = stl(time_series, 'periodic')
    Eta = exp(ma$time.series[,1])
    I = I*Eta

  
  ####alpha
  
  get_log_alpha_ratio = function(a, b, pi, A){
    lar23 <- numeric(train) 
    lar21 <- numeric(train) 
    
    lf = function(n, an) dnbinom(J[n], a[an], b/(I[n]+b), log = T)-dnbinom(J[n], a[2],  b/(I[n]+b), log = T)
    
    
    lar23[1]<-lf(1,3) + log(pi[3])-log(pi[2])
    lar21[1]<-lf(1,1) + log(pi[1])-log(pi[2])
    diff<-c(lar23[1], lar21[1], lar23[1]-lar21[1])
    
    for(i in 2:train){
      loglr23 = lf(i,3)
      loglr21 = lf(i,1)
      
      if(diff[1]>709){
        if(diff[3]>709){
          logarr23 = log(A[3,3])-log(A[3,2])
          logarr21 = log(A[3,1])-log(A[3,2])
        }else{
          logarr23 = log(A[1,3]+A[3,3]*exp(diff[3]))-log(A[1,2]+A[3,2]*exp(diff[3]))
          logarr21 = log(A[1,1]+A[3,1]*exp(diff[3]))-log(A[1,2]+A[3,2]*exp(diff[3]))
        }
      }else if(diff[2]>709){
        if(diff[3]>709){
          logarr23 = log(A[3,3])-log(A[3,2])
          logarr21 = log(A[3,1])-log(A[3,2])
        }else{
          logarr23 = log(A[1,3]+A[3,3]*exp(diff[3]))-log(A[1,2]+A[3,2]*exp(diff[3]))
          logarr21 = log(A[1,1]+A[3,1]*exp(diff[3]))-log(A[1,2]+A[3,2]*exp(diff[3]))
        }
      }else if(max(diff[1:2])<(-709)){
        logarr23 = log(A[2,3])-log(A[2,2])
        logarr21 = log(A[2,1])-log(A[2,2])
      }else{
        ar23 = lar23[i-1]
        ar21 = lar21[i-1]
        A13 = A[1,3]*exp(ar21)
        A23 = A[2,3]
        A33 = A[3,3]*exp(ar23)
        
        A12 = A[1,2]*exp(ar21)
        A22 = A[2,2]
        A32 = A[3,2]*exp(ar23)
        
        A11 = A[1,1]*exp(ar21)
        A21 = A[2,1]
        A31 = A[3,1]*exp(ar23)
        
        logarr23 = log(A13+A23+A33)-log(A12+A22+A32)
        logarr21 = log(A11+A21+A31)-log(A12+A22+A32)
      }
      
      lar23[i]<-(loglr23+logarr23)
      lar21[i]<-(loglr21+logarr21)
      
      diff<-c(lar23[i], lar21[i], lar23[i]-lar21[i])
    }
    
    return(cbind(lar21, lar23))
  }
  
  get_log_beta_ratio = function(a, b, A){
    lbr23 <- numeric(train)
    lbr21 <- numeric(train)
    
    lf = function(n, an) dnbinom(J[n], a[an], b/(I[n]+b), log = T)-dnbinom(J[n], a[2],  b/(I[n]+b), log = T)
    
    loglr23 = lf(train, 3)
    loglr21 = lf(train, 1)
    
    diff = c(loglr23, loglr21, loglr23-loglr21)
    if(diff[2]>709){
      if(diff[3]>709){
        lbr23[train]<-log(A[3,3])-log(A[2,1]+A[2,3])
        lbr21[train]<-log(A[1,3])-log(A[2,1]+A[2,3])
      }else{
        lbr23[train]<-log(A[3,1]+A[3,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
        lbr21[train]<-log(A[1,1]+A[1,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
      }
    }else if(diff[1]>709){
      if(diff[3]>709){
        lbr23[train] = log(A[3,3])-log(A[2,2])
        lbr21[train] = log(A[1,1])-log(A[1,2])
      }else{
        lbr23[train]<-log(A[3,1]+A[3,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
        lbr21[train]<-log(A[1,1]+A[1,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
      }
    }else{
      
      lbr23[train] = log(A[3,1]*exp(loglr21)+A[3,2]+A[3,3]*exp(loglr23))-log(A[2,1]*exp(loglr21)+A[2,2]+A[2,3]*exp(loglr23))
      lbr21[train] = log(A[1,1]*exp(loglr21)+A[1,2]+A[1,3]*exp(loglr23))-log(A[2,1]*exp(loglr21)+A[2,2]+A[2,3]*exp(loglr23))
    }
    
    
    for(i in (train-1):1){
      loglr23<-lf(i,3)
      loglr21<-lf(i,1)
      
      br23 = loglr23+lbr23[i+1]
      br21 = loglr21+lbr21[i+1]
      
      diff = c(br23, br21, br23-br21)
      
      if(diff[2]>709){
        if(diff[3]>709){
          lbr23[i]<-log(A[3,3])-log(A[2,1]+A[2,3])
          lbr21[i]<-log(A[1,3])-log(A[2,1]+A[2,3])
        }else{
          lbr23[i]<-log(A[3,1]+A[3,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
          lbr21[i]<-log(A[1,1]+A[1,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
        }
      }else if(diff[1]>709){
        if(diff[3]>709){
          lbr23[i] = log(A[3,3])-log(A[2,2])
          lbr21[i] = log(A[1,1])-log(A[1,2])
        }else{
          lbr23[i]<-log(A[3,1]+A[3,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
          lbr21[i]<-log(A[1,1]+A[1,3]*exp(diff[3]))-log(A[2,1]+A[2,3]*exp(diff[3]))
        }
      }else{
        
        lbr23[i] = log(A[3,1]*exp(br21)+A[3,2]+A[3,3]*exp(br23))-log(A[2,1]*exp(br21)+A[2,2]+A[2,3]*exp(br23))
        lbr21[i] = log(A[1,1]*exp(br21)+A[1,2]+A[1,3]*exp(br23))-log(A[2,1]*exp(br21)+A[2,2]+A[2,3]*exp(br23))
      }
      
    }
    return(cbind(lbr21, lbr23))
  }
  
  get_pij = function(logar, logbr, a, b, A){
    
    a2 = rep(0, train-1)
    a1 = (logar[-train,1])
    a3 = (logar[-train,2])
    
    b2 = rep(0, train-1)
    b1 = (logbr[-1,1])
    b3 = (logbr[-1,2])
    
    Jt = J[-1]
    It = I[-1]
    
    lnb = function(an) dnbinom(Jt, a[an], b/(It+b))
    
    p11_p = a1+b1+log(A[1,1])+lnb(1)
    p12_p = a1+b2+log(A[1,2])+lnb(2)
    p13_p = a1+b3+log(A[1,3])+lnb(3)
    
    p21_p = a2+b1+log(A[2,1])+lnb(1)
    p22_p = a2+b2+log(A[2,2])+lnb(2)
    p23_p = a2+b3+log(A[2,3])+lnb(3)
    
    p31_p = a3+b1+log(A[3,1])+lnb(1)
    p32_p = a3+b2+log(A[3,2])+lnb(2)
    p33_p = a3+b3+log(A[3,3])+lnb(3)
    
    pp = cbind(p11_p, p12_p, p13_p, 
               p21_p, p22_p, p23_p,
               p31_p, p32_p, p33_p)
    
    
    ppp = apply(pp,1,function(xx){
      if(all(is.infinite(xx))){
        xx<-ifelse(xx>0, exp(708), exp(-708))
      }
      if(max(xx)>708){
        mgap = max(xx)-708
        xx = xx-mgap
      }else if(min(xx)<(-708)){
        mgap = 708-max(xx)
        xx = xx+mgap
      }
      
      if(max(xx)>708){
        mgap = max(xx)-708
        xx = xx-mgap
      }
      
      return(exp(xx)/sum(exp(xx)))
    })
    ppp = t(ppp)
    
    for(k in c(1,4,7)){
      if(sum(ppp[,k:(k+2)])<exp(-708)){
        pp2 = pp[,k:(k+2)]
        if(max(pp2)>708){
          mgap = max(pp2)-708
          pp2 = pp2-mgap
        }else if(min(pp2)<(-708)){
          mgap = 708-max(pp2)
          pp2 = pp2+mgap
        }
        
        if(max(pp2)>708){
          mgap = max(pp2)-708
          pp2 = pp2-mgap
        }
        ppp2 = apply(pp2,1,function(xx){
          res = exp(xx)/sum(exp(xx))
          if(all(is.nan(res))){
            res = rep(0,3)
          }
          return(res)
        })
        ppp2 = t(ppp2)
        ppp[,k:(k+2)]<-ppp2
      }
    }
    
    
    return(ppp)
  }
  
  gap = 1000
  it.num = 1
  
  while((it.num<100 & gap>0.01)){
    
    logar = get_log_alpha_ratio(a, b, pi, A)
    logbr = get_log_beta_ratio(a, b, A)
    
    pb = logar+logbr
    
    ps = apply(pb, 1, function(x){
      diff = x[1]-x[2]
      if(x[1]>709){
        if(diff>709){
          y = c(1,0,0)
        }else{
          y = c(exp(diff)/(1+exp(diff)),0,1/(1+exp(diff)))
        }
      }else if(x[2]>709){
        if(diff<(-709)){
          y = c(0,0,1)
        }else if(diff>709){
          y = c(1,0,0)
        }else{
          y = c(exp(diff)/(1+exp(diff)),0,1/(1+exp(diff)))
        }
      }else{
        y = exp(c(x[1],0,x[2]))/sum(exp(c(0,x)))
      }
    })
    
    ps = t(ps)
    p1 = ps[,1]
    p2 = ps[,2]
    p3 = ps[,3]
    
    
    pij = get_pij(logar,logbr,a, b, A)
    
    newA = matrix(0,3,3)
    cols = colSums(pij)
    
    newA[1,1:3] = cols[1:3]/sum(cols[1:3])
    newA[2,1:3] = cols[4:6]/sum(cols[4:6])
    newA[3,1:3] = cols[7:9]/sum(cols[7:9])

    
    newpi = c(p1[1], p2[1], p3[1])
    newpi = ifelse(newpi==0,exp(-706), ifelse(newpi==1, 1-2*exp(-706), newpi))
    
    
    maxf = function(r,b,p){
      res = lgamma(r/b+J)-lgamma(r/b)-r/b*log(b)-r/b*log(1+I*b)
      return(sum(p*res))
    }

    willest = apply(ps,2,function(x) sum(x)>1)
    

    newb = ifelse(willest[2],optimize(function(x) -maxf(a[1], x, p1)-maxf(x,x,p2)-maxf(a[3],x,p3), c(a[1]*0.9, a[3]*1.1))$minimum, b)
    newa1 = ifelse(willest[1],optimize(function(x) -maxf(x,newb,p1), c(0.5, newb*0.99))$minimum, a[1])
    newa3 = ifelse(willest[3], optimize(function(x) -maxf(x,newb,p3), c(newb*1.01, 1.5*newb))$minimum, a[3])
    
    mgap = 10
    kk = 1
    while(mgap>0.01 & kk<50){
        newb = ifelse(willest[2],optimize(function(x) -maxf(a[1], x, p1)-maxf(x,x,p2)-maxf(a[3],x,p3), c(a[1]*0.9, a[3]*1.1))$minimum, b)
        newa1 = ifelse(willest[1],optimize(function(x) -maxf(x,newb,p1), c(0.5, newb*0.99))$minimum, a[1])
        newa3 = ifelse(willest[3], optimize(function(x) -maxf(x,newb,p3), c(newb*1.01, 1.5*newb))$minimum, a[3])

        mgap2 = abs(b-newb)
        if(mgap2>mgap){
            kk= 50
            newb = b
            newa1 = a[1]
            newa3 = a[3]
        }else{
            kk <- kk+1
            mgap = mgap2
        }
    }

    
    
    newa = c(newa1, newb, newa3)
    
    Agap = max(abs(A-newA))
    pigap = max(abs(pi-newpi))
    abgap = abs(b-newb)
    
    gap = max(Agap, pigap, abgap)
    
    if(!is.nan(gap)){
      A <- newA
      pi <- newpi
      a <- newa
      b <- newb
    }

    it.num = it.num+1
  }
  return(list(a = a, b=b, pi = pi, A = A, p1 = p1, p2 = p2, p3 = p3, it.num = it.num, Ieta = I))
}


#### online_fdr_ontime2

In [127]:
online_fdr_ontime2 = function(loc, a = c(450,500,550), b=500, pi=c(0.01,0.98,0.01), A=matrix(0.01,3,3)+diag(0.98,3), window = 30, start = 50){
  
  train = window
  J = get(paste0(loc,'_J'))
  J = J[!is.na(J)]; J[J==0] <- 1
  I = n_rolling2(J,7)
  st = which(I>0)[1]+start
  end = st + 365*2
  
  J = J[st:end]
  I = I[(st-1):(end-1)]
  
  dt = get(loc)$date[st:end]
  
  m2_res = m2_state3(loc,train=window, a=a, b=b, pi, A, start = start)
  
  p1 = m2_res$p1
  p2 = m2_res$p2
  p3 = m2_res$p3
  
  Ieta = numeric(length(J))
    Ieta[1:train] <- m2_res$Ieta[1:train]
    
  a = m2_res$a
  a_track = matrix(NA, nc = 3, nr = length(J))
  a_track[1:train,]<-rep(a, each = train)
  
  A_track = matrix(NA, nc = 9, nr = nrow(a_track))
  A_track[1:train,]<-rep(m2_res$A, each = train)
  
  blfdr = matrix(NA, nr = length(J), nc = 3)
  blfdr[1:train, 1]<-p1
  blfdr[1:train, 2]<-p2
  blfdr[1:train, 3]<-p3
  
  rej1 = logical(length(J))
  rej1[1:train] = offline_fdr(blfdr[1:train,1]+blfdr[1:train,2])
  
  maxl = (nrow(a_track)-window)
  for(i in 1:maxl){
    training = window
    aa = a_track[window+i-1,]
    m2_res = m2_state3(loc,train=training, a=aa, b=aa[2], start = start+i)

      Ieta[train+i] = m2_res$Ieta[train]

    a = m2_res$a; a_track[window+i,]<-a
    A = m2_res$A; A_track[window+i,]<-A 
    pi = c(m2_res$p1[2], m2_res$p2[2], m2_res$p3[2])
    pi = ifelse(pi==0,exp(-706), ifelse(pi==1, 1-2*exp(-706), pi))
    
    blfdr[i+window,] <- c(m2_res$p1[train], m2_res$p2[train], m2_res$p3[train])
    
    clfdr = m2_res$p1+m2_res$p2
    rej1[window+i] = (offline_fdr(clfdr))[window]
  }
  
  l = sum(!is.na(blfdr[,1]))
  rej1 = rej1[1:l]
  a_track = a_track[1:l,]
  A_track = A_track[1:l,]
  blfdr = blfdr[1:l,]
  rej1[1:window]<-F
  rseq = a_track[,3]/a_track[,2]-1
  clfdr = rowSums(blfdr[,1:2])
  rej2 = online_fdr(clfdr, rej1, window)

  
  rej = rej1 & rej2
  
  
  
  
  return(list(clfdr=clfdr, rej = rej, a_track = a_track, A_track = A_track, blfdr = blfdr, J = J, date = dt, Ieta = Ieta))
}


### Online FDR

In [72]:
p_values = function(nation){
    
    J = get(paste0(nation,'_J'))
    J = J[!is.na(J)]; J[J==0] <- 1
    I = n_rolling2(J,7)
    st = which(I>0)[1]+50
    end = st + 365*2

    
    J = J[st:end]
    I = I[(st-1):(end-1)]
    
    pval = sapply(1:length(J), function(i) ppois(J[i], lambda = I[i], lower.tail = FALSE))
    return(pval)
}

In [81]:
p_values2 = function(J, I){
    pval = sapply(1:length(J), function(i) ppois(J[i], lambda = I[i], lower.tail = FALSE))
    return(pval)
}

### Drawing Plot
This results in three plots:
1. raw number of Jt per 10000 which shows rejected days of the two models 
2. log number of Jt with rejected days of Model 1 (Red)
3. log number of Jt with rejected days of Model 2 (Blue)

In [17]:
draw_plot = function(mod1, mod2, pngname, maxdt = NA, train = 30){
    J = mod1$J
        if(is.na(maxdt)){
        maxdt = length(J)
        }
    I = mid_rolling(J)[1:maxdt]
    J = J[1:maxdt]
  
    rej1 = mod1$rej
    rej2 = mod2$rej
    dt = mod1$date


    png(paste0(pngname,'_raw.png'), height = 400, width =  1500)
    plot((I/10000), type = 'l', xlab = "", ylab = "", xaxt = 'n', cex.axis = 1.5)
    lines((J/10000), col = 'grey')
    seqq = 1
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%Y"), cex.axis = 1.5)
    seqq = grep('01-01', dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%Y %b"), cex.axis = 1.5)
    seqq = grep(c('05-01'), dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%b"), cex.axis = 1.5)
    seqq = grep(c('09-01'), dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%b"), cex.axis = 1.5)
  
  
    lines((I/10000))
    abline(v = which(rej1), col = rgb(255/255,51/255,51/255,0.08), lwd = 4)
    abline(v = which(rej2), col = rgb(51/255,51/255,255/255,0.08), lwd = 4)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)
    dev.off()
    
    png(paste0(pngname,'_mod1log.png'), height = 600, width =  1500)
    plot(log(I), type = 'l', xlab = "", ylab = "", xaxt = 'n', cex.axis = 1.5)
    lines(log(J), col = 'grey')
    seqq = 1
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%Y"), cex.axis = 1.5)
    seqq = grep('01-01', dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%Y %b"), cex.axis = 1.5)
    seqq = grep(c('05-01'), dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%b"), cex.axis = 1.5)
    seqq = grep(c('09-01'), dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%b"), cex.axis = 1.5)
  
  
    lines(log(I))
    abline(v = which(rej1), col = rgb(255/255,51/255,51/255,0.08), lwd = 4)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)
    dev.off()

    png(paste0(pngname,'_mod2log.png'), height = 600, width =  1500)
    plot(log(I), type = 'l', xlab = "", ylab = "", xaxt = 'n', cex.axis = 1.5)
    lines(log(J), col = 'grey')
    seqq = 1
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%Y"), cex.axis = 1.5)
    seqq = grep('01-01', dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%Y %b"), cex.axis = 1.5)
    seqq = grep(c('05-01'), dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%b"), cex.axis = 1.5)
    seqq = grep(c('09-01'), dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%b"), cex.axis = 1.5)
  
  
    lines(log(I))
    abline(v = which(rej2), col = rgb(51/255,51/255,255/255,0.08), lwd = 4)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)
    dev.off()
  
  
  
  print(paste0('Drawing Plots of ',pngname,' has been completed.'))
}

In [18]:
draw_plot1 = function(mod1, mod2, pngname, maxdt = NA, train = 30){
    J = mod1$J
        if(is.na(maxdt)){
        maxdt = length(J)
        }
    I = mid_rolling(J)[1:maxdt]
    J = J[1:maxdt]
  
    rej1 = mod1$rej
    rej2 = mod2$rej
    dt = mod1$date


    png(paste0(pngname,'.png'), height = 700, width = 1500)
    layout(mat = matrix(c(1,2,3), nr = 3), heights = c(500,50,50), widths = c(1500))
    par(mar = c(3,5,4,5))
    plot(log(I), type = 'l', xlab = "", ylab = "", xaxt = 'n', cex.axis = 1.5)
    lines(log(J), col = 'grey', type = 'b', cex =0.2, pch = 19)
    seqq = 1
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%Y"), cex.axis = 1.5)
    seqq = grep('01-01', dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%Y %b"), cex.axis = 1.5)
    seqq = grep(c('05-01'), dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%b"), cex.axis = 1.5)
    seqq = grep(c('09-01'), dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%b"), cex.axis = 1.5)
    lines(log(I))

    par(mar = c(1,5,1,5))
    plot(log(I), type = 'n', xlab = "", ylab = "", xaxt = 'n', yaxt = "n", cex.axis = 1.5)
    abline(v = which(rej1), col = rgb(255/255,51/255,51/255,0.5), lwd = 4)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)

    par(mar = c(1,5,1,5))
    plot(log(I), type = 'n', xlab = "", ylab = "", xaxt = 'n', yaxt = "n", cex.axis = 1.5)
    abline(v = which(rej2), col = rgb(51/255,51/255,255/255,0.5), lwd = 4)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)
    dev.off()
  
  
  
  print(paste0('Drawing Plots of ',pngname,' has been completed.'))
}

In [142]:
draw_plot2 = function(mod1, mod2, pngname, train = 30){
      
    J = get(paste0(pngname,'_J'))
    J = J[!is.na(J)]; J[J==0] <- 1
    I = n_rolling2(J,7)
    st = which(I>0)[1]+50
    end = st + 365*2
  
    J = J[st:end]
    I = I[(st-1):(end-1)]
    maxdt = length(I)
    #J = mod1$J
    #    if(is.na(maxdt)){
    #    maxdt = length(J)
    #    }
    #I = mid_rolling(J)[1:maxdt]
    is_inc = (c(diff(I),0)>0) & (c(diff(I,2),0,0)>0)
    is_inc[1:train] = FALSE
    #J = J[1:maxdt]
  
    rej1 = mod1$rej[1:maxdt]
    rej2 = mod2$rej[1:maxdt]
    
    datseq = log(J[-1]/I[-length(I)])
    nn = length(datseq)
    
    seasonal_online <- rep(0, nn)
    remainder_online <- rep(0, nn)
    window = 30
    
    for (i in (window+1):nn) {
      slice_data <- ts(datseq[(i-window):i], frequency = 7)
      stl_result <- stl(slice_data, s.window="periodic", t.degree = 1, s.degree = 1)
      
      seasonal_online[i] <- tail(stl_result$time.series[, 1], 1)
      remainder_online[i] <- tail(stl_result$time.series[, 3], 1)
    }
    
    eta = exp(seasonal_online + remainder_online)

    II = I
    II[-1] = II[-1]*eta

    p = p_values2(J, II)
    lord = LORD(p)$R
    saffron = SAFFRON(p)$R
    addis = ADDIS(p)$R
    lord[1:train] = 0
    saffron[1:train] = 0
    addis[1:train] = 0
    
    dt = mod1$date[1:maxdt]


    png(paste0(pngname,'.png'), height = 700, width = 1500, family = "Times")
    layout(mat = matrix(c(1,2,3,4,5,6), nr = 6), heights = c(500,50,50,30,30,30), widths = c(1500))
    par(mar = c(3,11,4,3))
    plot(log(I), type = 'l', xlab = "", ylab = "", xaxt = 'n', cex.axis = 2)
    #abline(v = which(is_inc), col = rgb(255/255, 204/255, 103/255, 0.3), lwd = 2)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)
    lines(log(J), col = 'black', type = 'b', cex =0.2, pch = 19)
    seqq = 1
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%Y"), cex.axis = 1.5)
    seqq = grep('01-01', dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%Y %b"), cex.axis = 1.5)
    seqq = grep(c('05-01'), dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%b"), cex.axis = 1.5)
    seqq = grep(c('09-01'), dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%b"), cex.axis = 1.5)
    lines(log(I))
    
    
    par(mar = c(1,11,1,3))
    plot(log(I), type = 'n', xlab = "", ylab = "", xaxt = 'n', yaxt = 'n',cex.axis = 3)
    text(x = par("usr")[1] - (par("usr")[2] - par("usr")[1]) * 0.06, y = mean(par("usr")[3:4]), 
     labels = "Model 1", srt = 0, adj = 0, xpd = TRUE, cex = 2)
    #abline(v = which(is_inc), col =  rgb(255/255, 204/255, 103/255, 0.3), lwd =2)
    abline(v = which(rej1), col = rgb(255/255,51/255,51/255,1), lwd = 6)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)

    par(mar = c(1,11,1,3))
    plot(log(I), type = 'n', xlab = "", ylab = "", xaxt = 'n', yaxt = 'n',cex.axis = 3)
    text(x = par("usr")[1] - (par("usr")[2] - par("usr")[1]) * 0.06, y = mean(par("usr")[3:4]), 
     labels = "Model 2", srt = 0, adj = 0, xpd = TRUE, cex = 2)
    #abline(v = which(is_inc), col =  rgb(255/255, 204/255, 103/255, 0.3), lwd = 2)
    abline(v = which(rej2), col = rgb(51/255,51/255,255/255,1), lwd = 6)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)

    par(mar = c(1,11,1,3))
    plot(log(I), type = 'n', xlab = "", ylab = "", xaxt = 'n', yaxt = 'n',cex.axis = 3)
    text(x = par("usr")[1] - (par("usr")[2] - par("usr")[1]) * 0.06, y = mean(par("usr")[3:4]), 
     labels = "LORD", srt = 0, adj = 0, xpd = TRUE, cex = 2)
    #abline(v = which(is_inc), col =  rgb(255/255, 204/255, 103/255, 0.3), lwd = 2)
    abline(v = which(lord==1), col = "darkgreen", lwd = 4)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)

    par(mar = c(1,11,1,3))
    plot(log(I), type = 'n', xlab = "", ylab = "", xaxt = 'n', yaxt = 'n',cex.axis = 3)
    text(x = par("usr")[1] - (par("usr")[2] - par("usr")[1]) * 0.06, y = mean(par("usr")[3:4]), 
     labels = "SAFFRON", srt = 0, adj = 0, xpd = TRUE, cex = 2)
    #abline(v = which(is_inc), col =  rgb(255/255, 204/255, 103/255, 0.3), lwd = 2)
    abline(v = which(saffron==1), col = "darkgreen", lwd = 4)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)

    par(mar = c(1,11,1,3))
    plot(log(I), type = 'n', xlab = "", ylab = "", xaxt = 'n', yaxt = 'n',cex.axis = 3)
    text(x = par("usr")[1] - (par("usr")[2] - par("usr")[1]) * 0.06, y = mean(par("usr")[3:4]), 
     labels = "ADDIS", srt = 0, adj = 0, xpd = TRUE, cex = 2)
    #abline(v = which(is_inc), col =  rgb(255/255, 204/255, 103/255, 0.3), lwd = 2)
    abline(v = which(addis==1), col = "darkgreen", lwd = 4)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)
    
    dev.off()
  
  
  
  print(paste0('Drawing Plots of ',pngname,' has been completed.'))
}

In [145]:
draw_plot3 = function(mod1, mod2, pngname, train = 30){
      
    J = get(paste0(pngname,'_J'))
    J = J[!is.na(J)]; J[J==0] <- 1
    I = n_rolling2(J,7)
    st = which(I>0)[1]+50
    end = st + 365*2
  
    J = J[st:end]
    I = I[(st-1):(end-1)]
    maxdt = length(I)
    #J = mod1$J
    #    if(is.na(maxdt)){
    #    maxdt = length(J)
    #    }
    #I = mid_rolling(J)[1:maxdt]
    is_inc = (c(diff(I),0)>0) & (c(diff(I,2),0,0)>0)
    is_inc[1:train] = FALSE
    #J = J[1:maxdt]
  
    rej1 = mod1$rej[1:maxdt]
    rej2 = mod2$rej[1:maxdt]
    
    datseq = log(J[-1]/I[-length(I)])
    nn = length(datseq)
    
    seasonal_online <- rep(0, nn)
    remainder_online <- rep(0, nn)
    window = 30
    
    for (i in (window+1):nn) {
      slice_data <- ts(datseq[(i-window):i], frequency = 7)
      stl_result <- stl(slice_data, s.window="periodic", t.degree = 1, s.degree = 1)
      
      seasonal_online[i] <- tail(stl_result$time.series[, 1], 1)
      remainder_online[i] <- tail(stl_result$time.series[, 3], 1)
    }
    
    eta = exp(seasonal_online + remainder_online)

    II = I
    II[-1] = II[-1]*eta

    p = p_values2(J, II)
    lord = LORD(p)$R
    saffron = SAFFRON(p)$R
    addis = ADDIS(p)$R
    lord[1:train] = 0
    saffron[1:train] = 0
    addis[1:train] = 0
    
    dt = mod1$date[1:maxdt]


    png(paste0(pngname,'.png'), height = 700, width = 1500, family = "Times")
    layout(mat = matrix(c(1,2,3), nr = 3), heights = c(500,50,50), widths = c(1500))
    par(mar = c(3,11,4,3))
    plot(log(I), type = 'l', xlab = "", ylab = "", xaxt = 'n', cex.axis = 2)
    abline(v = which(is_inc), col = rgb(255/255, 204/255, 103/255, 0.3), lwd = 2)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)
    lines(log(J), col = 'black', type = 'b', cex =0.2, pch = 19)
    seqq = 1
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%Y"), cex.axis = 1.5)
    seqq = grep('01-01', dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%Y %b"), cex.axis = 1.5)
    seqq = grep(c('05-01'), dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%b"), cex.axis = 1.5)
    seqq = grep(c('09-01'), dt)
    axis(1, at = seqq, labels = format(mod1$date[seqq],"%b"), cex.axis = 1.5)
    lines(log(I))
    
    
    par(mar = c(1,11,1,3))
    plot(log(I), type = 'n', xlab = "", ylab = "", xaxt = 'n', yaxt = 'n',cex.axis = 3)
    text(x = par("usr")[1] - (par("usr")[2] - par("usr")[1]) * 0.06, y = mean(par("usr")[3:4]), 
     labels = "Model 1", srt = 0, adj = 0, xpd = TRUE, cex = 2)
    abline(v = which(is_inc), col =  rgb(255/255, 204/255, 103/255, 0.3), lwd =2)
    abline(v = which(rej1), col = rgb(255/255,51/255,51/255,1), lwd = 6)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)

    par(mar = c(1,11,1,3))
    plot(log(I), type = 'n', xlab = "", ylab = "", xaxt = 'n', yaxt = 'n',cex.axis = 3)
    text(x = par("usr")[1] - (par("usr")[2] - par("usr")[1]) * 0.06, y = mean(par("usr")[3:4]), 
     labels = "Model 2", srt = 0, adj = 0, xpd = TRUE, cex = 2)
    abline(v = which(is_inc), col =  rgb(255/255, 204/255, 103/255, 0.3), lwd = 2)
    abline(v = which(rej2), col = rgb(51/255,51/255,255/255,1), lwd = 6)
    abline(v = 1:train, col = rgb(0,0,0,0.05), lwd = 4)
    
    dev.off()
  
  
  
  print(paste0('Drawing Plots of ',pngname,' has been completed.'))
}

In [20]:
par("usr")

[1] 0 1 0 1

### Data analysis

In [21]:
continent = c('Asia','Europe','Africa','North America','South America','Oceania')
max_country = character(length(continent))
names(max_country) = continent
new_tests<-ifelse(is.na(data$new_tests),0, data$new_tests)
new_cases = ifelse(is.na(data$new_cases),0, data$new_cases)
positive_rate <- ifelse(new_tests==0, 0, new_cases/new_tests)

max_country_tbl = data.frame(continent = continent, country = NA, population = NA, pr = NA)
for(cont in continent){
  
  pos = data.frame(location = data$location[data$continent==cont], population = data$population[data$continent==cont],
                   pr = positive_rate[data$continent==cont], 
                   ntest = new_tests[data$continent==cont])
  tbl = pos %>% group_by(location) %>% summarise(cnt = sum(!is.na(pr)),N = unique(population) , ntest = sum(ntest/population),m = mean(pr, na.rm = T))
  candi = which(tbl$N>10000000 & tbl$ntest>1)
  if(length(candi)==0){
    max_country[cont]<-NA
  }else{
    mini = which.max(unlist(tbl[candi,'ntest']))
    final = tbl$location[candi[mini]]
    max_country[cont]<-final
    max_country_tbl[max_country_tbl$continent == cont, 'country']<-final
    max_country_tbl[max_country_tbl$continent == cont, 'population']<-tbl$N[candi[mini]]
    max_country_tbl[max_country_tbl$continent == cont, 'pr']<-tbl$ntest[candi[mini]]
  }
}
max_country = max_country[!is.na(max_country)]

In [22]:
max_country

Asia           Europe    North America    South America 
   "South Korea" "United Kingdom"  "United States"          "Chile" 
         Oceania 
     "Australia"

In [56]:
data_of("South Korea", "south_korea")

[1] "south_korea has been extracted"


In [57]:
mod1 = online_fdr_ontime1("south_korea")
mod2 = online_fdr_ontime2("south_korea")

[1] 1
[1] 346.5594 500.0000 634.7630
[1] 2
[1] 346.5594 500.0000 634.7630
[1] 3
[1] 346.5594 500.0000 634.7630
[1] 4
[1] 346.5594 500.0000 634.7630
[1] 5
[1] 346.5594 500.0000 634.7630
[1] 6
[1] 346.5594 500.0000 634.7630
[1] 7
[1] 346.5594 500.0000 634.7630
[1] 8
[1] 346.5594 500.0000 634.7630
[1] 9
[1] 346.5594 500.0000 634.7630
[1] 10
[1] 346.5594 500.0000 634.7630
[1] 11
[1] 346.5594 500.0000 634.7630
[1] 12
[1] 346.5594 500.0000 634.7630
[1] 13
   lar21             lar23 
346.0010 500.0000 586.5423 
[1] 14
[1] 346.0010 500.0000 586.5423
[1] 15
   lar21             lar23 
344.4547 500.0000 586.5423 
[1] 16
[1] 344.4547 500.0000 586.5423
[1] 17
[1] 344.4547 500.0000 586.5423
[1] 18
   lar21             lar23 
327.1636 500.0000 586.5423 
[1] 19
[1] 327.1636 500.0000 586.5423
[1] 20
[1] 327.1636 500.0000 586.5423
[1] 21
[1] 327.1636 500.0000 586.5423
[1] 22
[1] 327.1636 500.0000 586.5423
[1] 23
[1] 327.1636 500.0000 586.5423
[1] 24
[1] 327.1636 500.0000 586.5423
[1] 25
[1] 327.1636 50

In [105]:
draw_plot2(mod1, mod2, nation)

[1] "Drawing Plots of south_korea has been completed."


In [108]:
for(country in max_country){
    nation = gsub(" ", "_", tolower(country))
    data_of(country, nation)
    print(paste0('-----------------', nation, '---------------------'))
    print('mod1 is now processing')
    mod1 = online_fdr_ontime1(loc = nation, start = 50)
    print('mod2 is now processing')
    mod2 = online_fdr_ontime2(loc = nation, start = 50)
  
    assign(paste0(nation,'_mod1'), mod1)
    assign(paste0(nation,'_mod2'), mod2)
}


[1] "south_korea has been extracted"
[1] "-----------------south_korea---------------------"
[1] "mod1 is now processing"
[1] "mod2 is now processing"
[1] 1
[1] 346.5594 500.0000 634.7630
[1] 2
[1] 346.5594 500.0000 634.7630
[1] 3
[1] 346.5594 500.0000 634.7630
[1] 4
[1] 346.5594 500.0000 634.7630
[1] 5
[1] 346.5594 500.0000 634.7630
[1] 6
[1] 346.5594 500.0000 634.7630
[1] 7
[1] 346.5594 500.0000 634.7630
[1] 8
[1] 346.5594 500.0000 634.7630
[1] 9
[1] 346.5594 500.0000 634.7630
[1] 10
[1] 346.5594 500.0000 634.7630
[1] 11
[1] 346.5594 500.0000 634.7630
[1] 12
[1] 346.5594 500.0000 634.7630
[1] 13
   lar21             lar23 
346.0010 500.0000 586.5423 
[1] 14
[1] 346.0010 500.0000 586.5423
[1] 15
   lar21             lar23 
344.4547 500.0000 586.5423 
[1] 16
[1] 344.4547 500.0000 586.5423
[1] 17
[1] 344.4547 500.0000 586.5423
[1] 18
   lar21             lar23 
327.1636 500.0000 586.5423 
[1] 19
[1] 327.1636 500.0000 586.5423
[1] 20
[1] 327.1636 500.0000 586.5423
[1] 21
[1] 327.1636 500

In [146]:
maxdtseq = integer(length(max_country))
names(maxdtseq) = max_country

for(country in max_country){
    print(country)
    nation = gsub(" ", "_", tolower(country))
    mod1 = get(paste0(nation, '_mod1'))
    mod2 = get(paste0(nation, '_mod2'))

    draw_plot3(mod1, mod2, nation)
}

[1] "South Korea"
[1] "Drawing Plots of south_korea has been completed."
[1] "United Kingdom"
[1] "Drawing Plots of united_kingdom has been completed."
[1] "United States"
[1] "Drawing Plots of united_states has been completed."
[1] "Chile"
[1] "Drawing Plots of chile has been completed."
[1] "Australia"
[1] "Drawing Plots of australia has been completed."


In [123]:
df = matrix(nr = length(max_country), nc = 11)
rownames(df) = max_country
colnames(df) = c("start","end","duration", "r1", "r2", "r3", "a1", "a2", "a3", "rej1", "rej2")
df = as.data.frame(df)

for(country in max_country){
    nation = gsub(" ", "_", tolower(country))
    maxdt = 365*2
    mod1 = get(paste0(nation, '_mod1'))
    mod2 = get(paste0(nation, '_mod2'))
    modr = mod1$r_track
    moda = mod2$a_track/mod2$a_track[,2]
    df[country,] = c(mod1$date[1], mod1$date[1]+maxdt+1, 365*2, apply(modr,2,median), apply(moda,2,median), sum(mod1$rej), sum(mod2$rej))
}

df[,1] = as.Date(df[,1])
df[,2] = as.Date(df[,2])

In [126]:
df

,start,end,duration,r1,r2,r3,a1,a2,a3,rej1,rej2
,<date>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
South Korea,2020-03-18,2022-03-19,730,0.8596031,1,1.242447,0.6196480,1,1.427629,243,126
United Kingdom,2020-03-25,2022-03-26,730,0.8314545,1,1.244371,0.6334821,1,1.500000,225,85
United States,2020-03-18,2022-03-19,730,0.8513435,1,1.161556,0.8733181,1,1.145014,170,181
Chile,2020-04-18,2022-04-19,730,0.8634992,1,1.146400,0.9010393,1,1.242297,166,97
Australia,2020-03-21,2022-03-22,730,0.7188152,1,1.464648,0.6130021,1,1.500000,199,137
